In [2]:
import openai
from openai import OpenAI

In [53]:
client = OpenAI()

In [ ]:
respone = client.chat.completions.create(
    model='gpt-3.5-turbo-1106',
    messages=[
        {"role": "user"},
        {"content": "hi course?"}
    ]
)

respone